In [41]:
#load modules to process imagery
import numpy as np
import gdal
import ogr
from skimage import exposure
from skimage.segmentation import quickshift
from skimage.segmentation import slic
import time
import scipy
import pandas as pd
import geopandas as gpd
from multiprocessing import Pool
import workers
import sklearn
from sklearn.ensemble import RandomForestClassifier

In [42]:
#Load GeoTiff
naip_fn = 'data/S3E115_clip.tif'

In [43]:
#set driver for geotiff
driverTiff = gdal.GetDriverByName('GTiff')

#open geotiff with gdal
naip_ds = gdal.Open(naip_fn)

#count the number of color bands in geotiff
nbands = naip_ds.RasterCount

#Create empty list
band_data = []

#export band numbers, and size of raster
print('bands', naip_ds.RasterCount, 'rows', naip_ds.RasterYSize, 'columns', naip_ds.RasterXSize)

#loop through the bands in nbands and append to empty band_data list
for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)

#stack arrays
band_data = np.dstack(band_data)

#export the shape of the data.  should match print statement above
print(band_data.shape)

bands 4 rows 10560 columns 10560
(10560, 10560, 4)


In [45]:
#rescale band data
img = exposure.rescale_intensity(band_data)

In [46]:
#create segments
segments = slic(img, n_segments=500000, compactness=0.1)

In [47]:
#Save segments to raster
segments_fn = 'data/segments_115.tif'
segments_ds = driverTiff.Create(segments_fn, naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(naip_ds.GetGeoTransform())
segments_ds.SetProjection(naip_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None

In [48]:
segment_ids = np.unique(segments)
objects = []

for id in segment_ids:
    object_features = workers.segment_features(img[segments == id])
    objects.append(object_features)

In [49]:
# open the points file to use for training data
train_fn = 'data/train_115.shp'
train_ds = ogr.Open(train_fn)
lyr = train_ds.GetLayer()

# create a new raster layer in memory
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())

# rasterize the training points
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)

# retrieve the rasterized data and print basic stats
data = target_ds.GetRasterBand(1).ReadAsArray()
print('min', data.min(), 'max', data.max(), 'mean', data.mean())

min 0 max 7 mean 6.246771694214877e-05


In [50]:
ground_truth = target_ds.GetRasterBand(1).ReadAsArray()

classes = np.unique(ground_truth)[1:]
print('class values', classes)

segments_per_class = {}

for klass in classes:
    segments_of_class = segments[ground_truth == klass]
    segments_per_class[klass] = set(segments_of_class)
    print("Training segments for class", klass, ":", len(segments_of_class))

intersection = set()
accum = set()

for class_segments in segments_per_class.values():
    intersection |= accum.intersection(class_segments)
    accum |= class_segments
assert len(intersection) == 0, "Segment(s) represent multiple classes"

class values [1 2 3 4 5 6 7]
Training segments for class 1 : 229
Training segments for class 2 : 238
Training segments for class 3 : 86
Training segments for class 4 : 525
Training segments for class 5 : 432
Training segments for class 6 : 147
Training segments for class 7 : 123


In [51]:
train_img = np.copy(segments)
threshold = train_img.max() + 1  # make the threshold value greater than any land cover class value

# all pixels in training segments assigned value greater than threshold
for klass in classes:
    class_label = threshold + klass
    for segment_id in segments_per_class[klass]:
        train_img[train_img == segment_id] = class_label
 
# training segments receive land cover class value, all other segments 0
train_img[train_img <= threshold] = 0
train_img[train_img > threshold] -= threshold

# create objects and labels for training data
training_objects = []
training_labels = []
for klass in classes:
    class_train_object = [v for i, v in enumerate(objects) if segment_ids[i] in segments_per_class[klass]]
    training_labels += [klass] * len(class_train_object)
    training_objects += class_train_object
 
classifier = RandomForestClassifier(n_jobs=-1)  # setup random forest classifier
classifier.fit(training_objects, training_labels)  # fit rf classifier
predicted = classifier.predict(objects)  # predict with rf classifier

# create numpy array from rf classifiation and save to raster
clf = np.copy(segments)
for segment_id, klass in zip(segment_ids, predicted):
    clf[clf == segment_id] = klass
 
mask = np.sum(img, axis=2)  # this section masks no data values
mask[mask > 0.0] = 1.0
mask[mask == 0.0] = -1.0
clf = np.multiply(clf, mask)
clf[clf < 0] = -9999.0
 
clfds = driverTiff.Create('data/classified_result_115.tif', naip_ds.RasterXSize, naip_ds.RasterYSize,
                          1, gdal.GDT_Int16)  # this section saves to raster
clfds.SetGeoTransform(naip_ds.GetGeoTransform())
clfds.SetProjection(naip_ds.GetProjection())
clfds.GetRasterBand(1).SetNoDataValue(-9999.0)
clfds.GetRasterBand(1).WriteArray(clf)
clfds = None
 
print('Done!')

Done!
